正确的 numpy 索引方式

In [37]:
import numpy as np

np.random.seed(42)

arr = np.zeros((2, 6, 5, 2))
len = [1, 2, 1, 1, 2]

brr = np.zeros((2, 6, 2))

for i in range(2):
    for j in range(5):
        for z in range(len[j]):
            k = 6
            indexs = np.random.choice(range(k), size=k, replace=False)
            weights = np.random.rand(k)
            proportions = np.random.rand(k)
            proportions = proportions / np.sum(proportions)
            arr[i, indexs, j, z] = proportions
            brr[i, indexs, z] = weights
# print(arr[0].round(3))
# print(arr[0].round(3))
print("=" * 30)
print(arr.shape)
print(arr[0].shape)
print(arr[0][:][:].shape)
# Wrong style, 不要使用链式索引
print("错误的索引结果：", arr[0][:][:][0].shape)
print("正确的索引结果：", arr[0, :, :, 0].shape)

# 验证 arr[i, :, j, z] 的和是否为 1
for i in range(2):
    for j in range(5):
        for z in range(len[j]):
            assert (arr[i, :, j, z].sum() - 1) < 1e-6


(2, 6, 5, 2)
(6, 5, 2)
(6, 5, 2)
错误的索引结果： (5, 2)
正确的索引结果： (6, 5)


In [38]:
print(arr)
print(brr)

[[[[0.24206172 0.        ]
   [0.35441711 0.04544662]
   [0.32606711 0.        ]
   [0.31589351 0.        ]
   [0.18874148 0.06643247]]

  [[0.00703702 0.        ]
   [0.2229045  0.25480601]
   [0.07141441 0.        ]
   [0.21896713 0.        ]
   [0.1183647  0.01935203]]

  [[0.28457888 0.        ]
   [0.03266015 0.01280617]
   [0.21786942 0.        ]
   [0.29461519 0.        ]
   [0.17914817 0.31314448]]

  [[0.06215869 0.        ]
   [0.13786291 0.16391127]
   [0.01648042 0.        ]
   [0.01166609 0.        ]
   [0.21349326 0.01565143]]

  [[0.07259026 0.        ]
   [0.16825016 0.33862777]
   [0.33592276 0.        ]
   [0.14835334 0.        ]
   [0.21256155 0.1935598 ]]

  [[0.33157343 0.        ]
   [0.08390516 0.18440217]
   [0.03224588 0.        ]
   [0.01050474 0.        ]
   [0.08769084 0.3918598 ]]]


 [[[0.13964965 0.        ]
   [0.00325198 0.3455392 ]
   [0.21980893 0.        ]
   [0.26692273 0.        ]
   [0.27226796 0.225328  ]]

  [[0.29492858 0.        ]
   [0.238910

numpy 的 `axis` 分析：沿着对应 axis 轴做 `sum` 等操作，对应维度消失

In [4]:
T = np.array([ 
    [[[1, 2]], [[3, 4]], [[5, 6]]],   # 第0个“大块”
    [[[7, 8]], [[9, 10]], [[11, 12]]] # 第1个“大块”
])
print("T.shape:", T.shape) # 输出: (2, 3, 1, 2)

# 现在计算 np.sum(T, axis=(1, 3))
result = np.sum(T, axis=(1, 3))
print("Result shape:", result.shape) # 输出: (2, 1)
print("Result:\n", result)

T.shape: (2, 3, 1, 2)
Result shape: (2, 1)
Result:
 [[21]
 [57]]


arr 提供了在 i 地块 index 物品的 j 年的 z 季度的比例, brr 提供在 i 地块 index 物品 z 季度的价格，请执行矩阵乘法得到所有 i 地块 j 年的总收入 result[i, j]


计算 result[i, j]，即各地块各年份的 总收入：
$$
\text{result}[i, j] = \sum_{\text{index}} \sum_{z} \left( \text{arr}[i, \text{index}, j, z] \times \text{brr}[i, \text{index}, z] \right)
$$

In [39]:
# 对(i, j) 维度进行求和
result = np.sum(arr * brr[:, :, np.newaxis, :], axis=(1, 3))

print(result.round(3))

result_manual = np.zeros((2, 5))
for i in range(2):
    for j in range(5):
        total = 0
        for index in range(6):
            for z in range(2):
                total += arr[i, index, j, z] * brr[i, index, z]
        result_manual[i, j] = total

assert np.allclose(result, result_manual, rtol=1e-3)  # 允许 0.001 的相对误差

[[0.495 1.412 0.697 0.648 1.153]
 [0.72  1.396 0.696 0.654 1.543]]


基本索引操作总是创建视图。

In [46]:
x = np.arange(10)
print("原来的 x:")
print(x)
y = x[1:3]
print("原来的 y:")
print(y)
print("修改 x[1:3]")
x[1:3] = [10, 9]
print("修改后的 x:")
print(x)
print("影响到 y:(因为 y 是 x 的一个视图而非拷贝)")
print(y)

原来的 x:
[0 1 2 3 4 5 6 7 8 9]
原来的 y:
[1 2]
修改 x[1:3]
修改后的 x:
[ 0 10  9  3  4  5  6  7  8  9]
影响到 y:(因为 y 是 x 的一个视图而非拷贝)
[10  9]


高级索引总是创建副本。

当选择对象 obj 是非元组序列对象、ndarray（数据类型为整数或布尔），或者包含至少一个序列对象或 ndarray（数据类型为整数或布尔）的元组时，会触发高级索引。高级索引有两种类型：整数索引和布尔索引。

In [49]:
import numpy as np
x = np.arange(9).reshape(3, 3)
print(x)

y = x[[1, 2]]
y

y.base is None

[[0 1 2]
 [3 4 5]
 [6 7 8]]


True